# Descarga y procesamiento de imágenes GOES-16 para generar recortes en color verdadero


@Tesis: Modelo de segmentación semántica de columnas de humo derivadas de incendios forestales en México

@Autor: Colvert Gomez Rubio

In [1]:
import warnings
warnings.filterwarnings('ignore')

### Librerías

In [2]:
from goes2go.data import goes_nearesttime
import pandas as pd
import numpy as np
from osgeo import gdal,osr
import cv2
import os
from shutil import rmtree

### Funciones

In [9]:
def extrae_datos_netcdf(ds, band):
    """
    Extrae la matriz de datos de la banda 'band' de un dataset xarray.core.dataset.Dataset.
    Args:
        ds (xarray.core.dataset.Dataset): Dataset de xarray.
        band (int): Número de la banda a extraer.
    Returns:
        numpy.ndarray: Matriz de datos de la banda especificada.
    """
    data = ds.variables['CMI_C' + str(band).zfill(2)][:].data    
    return data

def true_color_goes16(ds_netcdf):
    """
    Genera una imagen de color verdadero a partir de las bandas del dataset GOES-16.
    Args:
        ds_netcdf (xarray.core.dataset.Dataset): Dataset de xarray.
    Returns:
        numpy.ndarray: Imagen de color verdadero generada.
    """
    c01 = extrae_datos_netcdf(ds_netcdf, 1) 
    c02 = extrae_datos_netcdf(ds_netcdf, 2) 
    c03 = extrae_datos_netcdf(ds_netcdf, 3) 

    c01 = np.power(c01, 0.05)
    c02 = np.power(c02, 0.2)
    c03 = np.power(c03, 0.2)

    c01 = (c01 - np.nanmin(c01)) * 255 / (np.nanmax(c01) - np.nanmin(c01))
    c02 = (c02 - np.nanmin(c02)) * 255 / (np.nanmax(c02) - np.nanmin(c02))
    c03 = (c03 - np.nanmin(c03)) * 255 / (np.nanmax(c03) - np.nanmin(c03))

    verdeSint = 0.48358168 * c02 + 0.45706946 * c01 + 0.06038137 * c03
    true_color = (np.dstack((c02, verdeSint, c01))).astype('uint8') #red(b2), verdesint, blue(b1)
    
    return true_color


def blue_land_goes16(ds_netcdf):
    """
    Genera una imagen de paisaje azul a partir de las bandas del dataset GOES-16.
    Args:
        ds_netcdf (xarray.core.dataset.Dataset): Dataset de xarray.
    Returns:
        numpy.ndarray: Imagen de paisaje azul generada.
    """
    c01 = extrae_datos_netcdf(ds_netcdf, 1)
    c02 = extrae_datos_netcdf(ds_netcdf, 2)
    c03 = extrae_datos_netcdf(ds_netcdf, 3)
    
    c01 = np.power(c01, 0.2)
    c02 = np.power(c02, 0.4)
    c03 = np.power(c03, 0.4)

    c01 = (c01 - np.nanmin(c01))*255/(np.nanmax(c01)-np.nanmin(c01))
    c02 = (c02 - np.nanmin(c02))*255/(np.nanmax(c02)-np.nanmin(c02))
    c03 = (c03 - np.nanmin(c03))*255/(np.nanmax(c03)-np.nanmin(c03))

    blue_land = (np.dstack((c01,c02,c03))).astype('uint8') 
    
    return blue_land


def crea_tif_geos_rgb(ds, data, path, name):
    """
    Crea un GeoTIFF '.tif' en coordenadas geoestacionarias con los siguientes parámetros.
    Args:
        ds (xarray.core.dataset.Dataset): Dataset de xarray.
        data (numpy.ndarray): Matriz de datos a ser guardada en el GeoTIFF.
        path (str): Ruta del directorio donde se almacenará el archivo.
        name (str): Nombre del archivo GeoTIFF.
    """
    H = ds['goes_imager_projection'].perspective_point_height
    xmin = ds['x_image_bounds'][0].data * H
    xmax = ds['x_image_bounds'][1].data * H
    ymin = ds['y_image_bounds'][0].data * H
    ymax = ds['y_image_bounds'][1].data * H
    
    nx = data.shape[1]
    ny = data.shape[0]
    res_x = (xmax - xmin) / float(nx) 
    res_y = (ymax - ymin) / float(ny) 
    geotransform = (xmin, res_x, 0, ymin, 0, res_y)
    dst_ds = gdal.GetDriverByName('GTiff').Create(path + name, nx, ny, 3, gdal.GDT_Float32)
    
    dst_ds.SetGeoTransform(geotransform)   
    srs = osr.SpatialReference()        
    srs.ImportFromProj4('+proj=geos +h=35786023.0 +ellps=GRS80 +lat_0=0.0 +lon_0=-75.0 +sweep=x +no_defs')
    dst_ds.SetProjection(srs.ExportToWkt())
    
    for i in range(3):  # Iterar sobre las 3 bandas
        dst_ds.GetRasterBand(i+1).WriteArray(data[:,:,i])  # Escribir cada banda en su respectiva banda del archivo
    
    dst_ds.FlushCache()                  
    dst_ds = None
    
    
def recorta_tif(ds_netcdf, ds_tif, xmin_rec, ymin_rec, num_pix):
    """
    Recorta un GeoTIFF en las coordenadas recibidas.
    Args:
        ds_netcdf (xarray.core.dataset.Dataset): Dataset de xarray.
        ds_tif (osgeo.gdal.Dataset): Dataset del GeoTIFF a recortar.
        xmin_rec (float): Coordenada X mínima del recorte.
        ymin_rec (float): Coordenada Y mínima del recorte.
        num_pix (int): Número de píxeles del recorte.
    Returns:
        str: Nombre del archivo GeoTIFF recortado.
    """
    H = ds_netcdf['goes_imager_projection'].perspective_point_height
    xmin = ds_netcdf['x_image_bounds'][0].data * H
    xmax = ds_netcdf['x_image_bounds'][1].data * H
    ymin = ds_netcdf['y_image_bounds'][0].data * H
    ymax = ds_netcdf['y_image_bounds'][1].data * H
    nx = ds_tif.RasterXSize
    ny = ds_tif.RasterYSize
    res_x = (xmax - xmin) / float(nx) 
    res_y = (ymax - ymin) / float(ny)
    
    xmax_rec = xmin_rec + res_x * num_pix
    ymax_rec = ymin_rec - res_y * num_pix
    coordinates = [xmin_rec, ymax_rec, xmax_rec, ymin_rec]
    
    #print (coordinates)
    
    name_rec = ds_tif.GetDescription().replace(".tif","_r.tif")
    gdal.Translate(name_rec, ds_tif, options = gdal.TranslateOptions(projWin=coordinates, noData=np.nan))
    
    return name_rec


def crea_tif_geos_rgb_inv(data, geotransform, pathname):
    """
    Crea un GeoTIFF '.tif' en coordenadas geoestacionarias.
    Args:
        data (numpy.ndarray): Matriz de datos a ser guardada en el GeoTIFF.
        geotransform (tuple): Transformación geoespacial del GeoTIFF.
        pathname (str): Ruta completa del archivo GeoTIFF.
    """
    
    dst_ds = gdal.GetDriverByName('GTiff').Create(pathname, data.shape[0], data.shape[1], 3, gdal.GDT_Float32)
    
    dst_ds.SetGeoTransform(geotransform)   
    srs = osr.SpatialReference()        
    srs.ImportFromProj4('+proj=geos +h=35786023.0 +ellps=GRS80 +lat_0=0.0 +lon_0=-75.0 +sweep=x +no_defs')
    dst_ds.SetProjection(srs.ExportToWkt())
    
    for i in range(3):  # Iterar sobre las 3 bandas
        dst_ds.GetRasterBand(i+1).WriteArray(data[:,:,i])  # Escribir cada banda en su respectiva banda del archivo
    
    dst_ds.FlushCache()                  
    dst_ds = None

### Directorios

In [4]:
data_path = ".../data/"
temp_path = ".../temp/"
smoke_path = ".../smoke_png/"
geotransform_path = ".../geotransform/"

### Función Main

In [7]:
def main(fecha, rango_horas, frecuencia, coordenadas_inciales, numero_pixeles, geotransform):
    """
    Procesa datos satelitales y genera imágenes PNG a partir de archivos NetCDF.

    Parámetros:
    - fecha (str): Fecha en formato YYYY-MM-DD.
    - rango_horas (list): Lista que contiene el rango de horas en formato ["HH:mm", "HH:mm"].
    - frecuencia (str): Frecuencia de descarga de datos en formato de pandas frequency string (e.g., "1H", "30min").
    - coordenadas_inciales (list): Lista que contiene las coordenadas iniciales [latitud, longitud] para el recorte de la imagen.
    - numero_pixeles (int): Número de píxeles para el recorte de la imagen.
    - geotransform (bool): Indica si se debe actualizar el archivo CSV con información de geotransformación.

    Retorna:
    None
    """
    #Genera lista de horarios a descargar
    hours_list = list(pd.date_range(rango_horas[0], rango_horas[1], freq = frecuencia).strftime('%H:%M')) #"1H", "30min"

    for hour in hours_list:
        #Descarga netcdf4
        print("Procesando: " + fecha + " " + hour)
        ds_netcdf = goes_nearesttime(fecha + " " + hour, satellite=16, product="ABI", domain="C")
        ds_netcdf_path = data_path + str(ds_netcdf.path[0])
        
        #Genera compuesto rgb en un numpy.ndarray
        rgb = true_color_goes16(ds_netcdf)
        
        #Genera tif del rgb en geoestacionarias de CONUS en temp
        tif_conus_rgb_name = ds_netcdf.path[0].split('/')[-1][11:36] + "_tc.tif"
        crea_tif_geos_rgb(ds_netcdf, rgb, temp_path, tif_conus_rgb_name)
        
        #Recorta el tif a coordenadas dadas
        ds_tif = gdal.Open(temp_path + tif_conus_rgb_name) 
        tif_rec_rgb_name = recorta_tif(ds_netcdf, ds_tif, coordenadas_inciales[0], coordenadas_inciales[1], numero_pixeles)

        #Elimina netcdf4 de data
        ds_netcdf = None
        os.remove(ds_netcdf_path)
        rmtree(ds_netcdf_path[:32])
        
        #Elimina de temp el tif del rgb en geoestacionarias de CONUS
        ds_tif = None
        os.remove(temp_path + tif_conus_rgb_name)
        
        #Genera png del tif recortado del png en smoke_path
        #print("Gerando png...")
        img_rec_rgb = cv2.imread(tif_rec_rgb_name, cv2.IMREAD_UNCHANGED)
        png_name = tif_rec_rgb_name.replace(".tif", str(coordenadas_inciales[0]*coordenadas_inciales[1])[:5] + ".png").split("/")[-1]
        cv2.imwrite(smoke_path + png_name, img_rec_rgb)
        
        #Obtiene geotransformada del tif recortado en rgb
        ds_tif_rec = gdal.Open(tif_rec_rgb_name) 
        geotransform_rec = ds_tif_rec.GetGeoTransform()
        
        #Elimina de temp el tif recortado del rgb en geoestacionarias 
        ds_tif_rec = None
        os.remove(tif_rec_rgb_name)

        #Actualiza csv de geotransform
        if geotransform == True:
            ## Verificar si el archivo existe
            if os.path.isfile(os.path.join(geotransform_path, "geotransforms.csv")):
                # El archivo existe, abrirlo y agregar nuevas filas
                geotransform_df = pd.read_csv(os.path.join(geotransform_path, "geotransforms.csv"))
            else:
                # El archivo no existe, crear un nuevo DataFrame
                geotransform_df = pd.DataFrame(columns=['Date', 'Name', 'Geotransform'])
                
            nueva_fila = {'Date' : str(tif_rec_rgb_name.split("/")[-1][14:25]), 'Name' : png_name, 'Geotransform' : geotransform_rec} # creamos un diccionario
            geotransform_df = geotransform_df.append(nueva_fila, ignore_index=True)
            geotransform_df.to_csv(os.path.join(geotransform_path, "geotransforms.csv"), index = False)
        
        print("---------------------------------") 

### Ejecución

In [11]:
main(fecha = "2022-06-06", rango_horas = ["17:00","17:10"], frecuencia = "5min", coordenadas_inciales = [-2300000, 2380000], numero_pixeles = 100, geotransform = True)

Procesando: 2022-06-06 17:00
📦 Finished downloading [1] files to [C:\Users\colve\data\noaa-goes16\ABI-L2-MCMIPC].
📚 Finished reading [1] files into xarray.Dataset.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     